In [1]:
import pandas as pd
import numpy as np
import math
import re

import io

can_have_resource=["img_old","img_forensic_discrepency","img_metadata_discrepency","img_logical_discrepency","vid_old","vid_forensic_discrepency","vid_metadata_discrepency","vid_audio_discrepency","vid_logical_discrepency","txt_unreliable_news_content","txt_insufficient_verifiable_srcs","txt_has_clickbait","txt_poor_language","txt_crowds_distance_discrepency","txt_author_offers_little_evidence","txt_reliable_sources_disapprove",
"medium_is_blacklisted","medium_failed_factcheck_before","medium_has_poor_security","medium_whois_info_discrepency","medium_hosting_discrepency","medium_is_biased","medium_is_poorly_ranked","medium_is_not_liable","medium_lacks_flagging_means",
"src_lacks_proper_credentials","src_failed_factcheck_before","src_has_poor_writing_history","src_supported_by_trolls","src_difficult_to_locate"]

def process_pages(page):
      title_or_name="name"
      full_page=page
      if page=="claim":
        title_or_name="title"
      if page=="src":
        full_page="source"
      all_sections = pd.read_csv("q - "+page+"_review.csv")
      all_sections["sec_number"] = all_sections["sec_number"].astype(str)

      unique_sections=all_sections.sec_number.unique()

      tmp=all_sections[['sec_number','q_title']].groupby(['sec_number']).agg(['count'])
      sec_size={}
      variables={}
      for s in unique_sections:
          sec_size[s]=tmp.query("sec_number=='"+s+"'").iloc[0]['q_title']['count']
          
      progress_bar='''
      '''.format()

      max=len(unique_sections)
      last_sec=""
      controller_steps=""
      validation=""
      params="def "+page+"_review_params\n  params.require(:"+page+"_review).permit(:id"
    
      all_steps=""
      for index, row in all_sections.iterrows():
        try:
            if (len(str(row['q_short']))>3):
                all_steps=all_steps+","+"'"+str(row['sec_number'])+"'=>t('"+str(row['q_short'])+"')"
        except:
            print ("error here!")
      
      all_steps=all_steps.strip(",")
    
      for index, row in all_sections.iterrows():
          row['variable_name']=str(row['variable_name'])
          if last_sec!=row['sec_number']:
              last_sec=row['sec_number']
              vrs=""
              field=""
              buttons=""
              file_contents=""
              questions=""
              i=0
              req=""
          i=i+1
          
          verdict_note=""
          verdict_click=""
          if ("review_description" in row['variable_name'] and "note" not in row['variable_name']):
            verdict_note="<br><div id='verdict_note' style='border:2px solid black;margin:auto;width:70%;padding:10px;background-color:lightblue;display:none;'><b><%= t('note_provide_rationale') %></div><br>"

          if ("review_verdict" in row['variable_name']):
            verdict_click=", onClick: \"$('#verdict_note').show();\""
          
          note=""
          if (("review_started" in row['variable_name'] or "src_lacks_proper_credentials" in row['variable_name'] or "medium_is_blacklisted" in row['variable_name']) and "note" not in row['variable_name']):
            note="<br><div class='alert alert-success'><b><big><%= t('note') %>: </big></b><%== t('not_all_qs_needed', page:'"+page+"') %></div><br>"

          required=""
          if row['is_required']:
              req="<span class='required'>* <%= t('required') %></span>"
              validation=validation+"\nvalidates :"+row['variable_name']+", presence: true, if: -> { required_for_step?(:"+row['sec_number']+") }"
              required="<span class='required'>*</span>"
          if row['text_field_type']=="text_field":
              field="<%= f.text_field :"+row['variable_name']+", class: 'field-long' %>"
          elif row['text_field_type']=="textarea":
            if row['variable_name']=="note_review_sharing_mode":
              field="<%= f.text_area(:"+row['variable_name']+", size: '100x10', class: 'field-long field-textarea') %><br>"
            else:
              field="<%= f.text_area(:"+row['variable_name']+", size: '50x10', class: 'field-long field-textarea') %><br>"
          elif row['text_field_type']=="radio":
              options=row['options'].split(',')
              options_val=str(row['options_val']).split(' ')
              maxl=len(options)
              field="<br>"
              skip=""
              if (not row['stage_title'].startswith('assessing_') and not row['stage_title'].startswith('share_setting')):
                skip="<%= f.radio_button :"+row['variable_name']+", '', :checked => (@"+page+"_review."+row['variable_name']+".blank?) %> <%= t('skip_question') %><br>"
              for ix in range(maxl):
                  field=field+"<%= f.radio_button :"+row['variable_name']+", '"+str(options_val[ix])+"', :checked => (!@"+page+"_review."+row['variable_name']+".blank? && @"+page+"_review."+row['variable_name']+"=="+str(options_val[ix])+")"+verdict_click+" %> <%== t('"+options[ix]+"') %><br>"
              field=field+"<br>"+skip
          else:
              field="<%= f.hidden_field :"+page+"_review, :value => @"+page+"_review %>"

          resource_block=''
          if row['variable_name'] in can_have_resource:
              resource_block='''
                  <br>
                  <%
                    require 'digest/sha1';
                    current_code=Digest::SHA1.base64digest("@{p}_review."+"{v}")
                    has_resources=Resource.where("used_for_qs like ?", "%"+current_code+"%")
                  %>
                  <div class="panel-group" id="accordion1">
                  	<div class="panel panel-default">
                      	<div class="panel-heading">
                              	<a href="#Resources" data-toggle="collapse" data-parent="#accordion1" style="text-decoration: none">
                                  <b><%= t('resources') %> (<%= has_resources.size.to_s %>)</b> &nbsp; <span class="glyphicon glyphicon-plus"></span></a>
                          </div>
                          <div class="panel-collapse collapse" id="Resources">
                          	<div class="panel-body"><br>
                            <% if not has_resources.blank? %>
                                  <ul>
                                  <% has_resources.each do |r| %>
                                  <li><h6>- <%== try_resource(r.name,'{p}') %><%= link_to r.name, resource_path(r.id), target: "_blank" %></h6></li>
                                <% end %>
                                <% if (not @{p}.url.blank?) %>
                                      <hr><small><%== t('{p}')[0].upcase+t('{p}')[1..-1]+" URL: <input type='text' id='url_link' value='"+@{p}.url.to_s+"' style='background-color:lightgrey;width:75%'> <button onclick=copyToClipboard('#url_link')>"+t('copy_url')+"</button>" %></small>
                                <% end %>
                              </ul>
                              <hr><p>- <%= link_to t('add_additional_resource'), new_resource_path(ref: current_code), target: "_blank" %><br><br>
                              <small>- <%= t('check_current_resources1') %><b><%= link_to t('resources'), resources_path(), target: "_blank" %></b><%= t('check_current_resources2') %> <b><%= current_code %></b></small></p>
                          <% else %>
                              <% if (not @{p}.url.blank?) %>
                                   <hr><small><%== t('{p}')[0].upcase+t('{p}')[1..-1]+" URL: <input type='text' id='url_link' value='"+@{p}.url.to_s+"' style='background-color:lightgrey;width:75%'> <button onclick=copyToClipboard('#url_link')>"+t('copy_url')+"</button>" %></small>
                              <% end %>
                              <p><b><% t('no_associated_resource') %></b></p>
                              <hr><p>- <%= link_to t('add_resource_to_q'), new_resource_path(ref: current_code), target: "_blank" %><br><br>
                              <small>- <% t('associate_to_q') %> <% t('this_resource') %>: <b><%= current_code %></b></small></p>
                          <% end %>
                          </div>
                       </div>
                  </div>
                  <hr>
             '''.format(p=page,v=row['variable_name'])
          row['stage_title']="<%== t('"+row['stage_title']+"') %>"
          if (row['q_title']=="calculated_score_q_long"):
            row['q_title']="<%== t('"+row['q_title']+"', review_score: @"+page+"_review_score) %>"
          elif (row['q_title']=="claim_review_field"):
            row['q_title']="<%== t('"+row['q_title']+"') %><br><small><small><%== t('published_article_url') %></small></small>"
          else:
            row['q_title']="<%== t('"+row['q_title']+"') %>"
          
          questions=questions+'''
          {vn}<li><big><b><font color=blue>{question}</font></b></big>{required}<br>
          {field}
          {note}
          <br>
          {rb}
          </li>
              '''.format(vn=verdict_note,question=row['q_title'],required=required,field=field,note=note,rb=resource_block)

          if (row['variable_name']=="add_to_blockchain"):
            questions=re.sub(r'(<li>.+add_to_blockchain_q_long.+)',r'<% if !@claim.blockchain_tx.nil? && ENV["BLOCKCHAIN_ENABLED"]=="true" %>\n\1\n',questions)
            questions=questions+"<% end %>"
        
          if (not pd.isna(row['variable_name']) and row['variable_name']!='nan'):
              vrs=vrs+", :"+row['variable_name']
              params=params+", :"+row['variable_name']

          if (i==sec_size[row['sec_number']]):
              if (row["sec_number"]==unique_sections[0]):
                  buttons="<%= f.submit t('next_step'), :class => 'btn btn btn-primary' %>"
              elif (row["sec_number"]==unique_sections[max-1]):
                  buttons="<%= link_to t('previous_step'),previous_wizard_path+'?s=prev',:class=>'btn btn btn-primary',:style=>'color:white; background-color:#4B99AD' %>\n&nbsp;&nbsp;&nbsp;<%= f.submit t('finish'), :class => 'btn btn btn-primary' %>"
              else:
                  buttons='''
                        <% if !@first_step.present? or @first_step!=step %>
                            <%= f.submit t('previous_step'), :class => 'btn btn btn-primary' %>
                        <% end %>
                            <%= f.submit t('next_step'), :class => 'btn btn btn-primary' %>
                  '''.format()
                    
              file_content='''
         <!-- Heading -->
<div class="text-center mb-5">
  <div class="col-12 col-md-10 col-lg-8 mx-auto">
    <h4><%= t('{p}_r') %></h4>
    <h2 class="text-primary"> <%= link_to @{p}.{title_or_name}, {p}_path(@{p}.id) %></h2>
  </div>
</div>
<!-- Heading -->
<div class="container">
  <div class="row">
    <div class="col-3">
      <%= form_for @{p}_review, method: :put, url: wizard_path do |f| %>
      <div class="">
        <% 
            passed_step=false
            @all_steps.each do |key,value| %>
        <%= is_visible(key).html_safe %><div class="">
          <% if (step==File.basename(wizard_path(key)))
                    passed_step=true %>
          <div class="no-style"><b><%== value %></b></div>
          <% elsif passed_step %>
          <%= f.submit value, style: '', class: 'no-style' %>
          <% else %>
          <%= f.submit value, style: '', class: 'no-style' %>
          <% end %>
        </div>
      </div>
      <% end %>
    </div>
  </div>
  <div class="col-9">
    <% if f.object.errors.any? %>
    <div class="error_messages">
      <% f.object.errors.full_messages.each do |error| %>
      <p><%= error %></p>
      <% end %>
    </div>
    <% end %>
    <fieldset>
      <br><h6><strong>{st}</strong></h6>
      {qs}
      {b}
    </fieldset>
        <% if ENV['BLOCKCHAIN_ENABLED']!='true' %>
            <div style='float:right'><%= link_to t('del_review'), {p}_{p}_review_path(@{p}.id,@{p}_review), style: '', method: :delete, data: {{ confirm: t('warning_del') }} %></div>
        <% end %>
    <% end %>
  </div>
</div>
              '''.format(sn=row['stage_number'],st=row['stage_title'],qs=questions,b=buttons,p=page,fp=full_page,title_or_name=title_or_name)
              vrs=vrs.lstrip(", ")
              if (vrs):
                  controller_steps=controller_steps+"\n\t\twhen \""+row['sec_number']+"\"\n\t\t\t["+vrs+"]"
              with io.open("../app/views/"+page+"_review/steps/"+row['sec_number']+".html.erb", 'w', encoding='utf8') as f:
                  f.write(file_content)

      params=params.rstrip(",")
      params=params+")\n  end"
      print ("-------------------"+page+"_review--------------------------\n")
      print ("Add this Step Names line to models/"+page+"_review.rb:\n\n %w("+' '.join(unique_sections)+")\n"+validation)
      print ("\n\nAdd those lines to controllers/"+page+"_review_controller.rb: \n"+params+"\n\n")
      print ("\n\nAdd this line to controllers/"+page+"_review/steps_controller.rb inside def find_"+page+": \n\n@all_steps={"+all_steps+"}\n\n");
      print ("\n\nAdd those lines to controllers/"+page+"_review/steps_controller.rb: \n"+controller_steps+"\n\n")
    


In [2]:
process_pages("claim")

-------------------claim_review--------------------------

Add this Step Names line to models/claim_review.rb:

 %w(s1 s2 s3 s4 s5 s6 s7 s8 s9 s10 s11 s12 s13 s14 s15 s16 s17 s18 s19 s20 s21 s22)

validates :review_verdict, presence: true, if: -> { required_for_step?(:s21) }
validates :review_sharing_mode, presence: true, if: -> { required_for_step?(:s22) }


Add those lines to controllers/claim_review_controller.rb: 
def claim_review_params
  params.require(:claim_review).permit(:id, :img_review_started, :img_old, :note_img_old, :img_forensic_discrepency, :note_img_forensic_discrepency, :img_metadata_discrepency, :note_img_metadata_discrepency, :img_logical_discrepency, :note_img_logical_discrepency, :vid_review_started, :vid_old, :note_vid_old, :vid_forensic_discrepency, :note_vid_forensic_discrepency, :vid_metadata_discrepency, :note_vid_metadata_discrepency, :vid_audio_discrepency, :note_vid_audio_discrepency, :vid_logical_discrepency, :note_vid_logical_discrepency, :txt_review_sta

In [3]:
process_pages("medium")

-------------------medium_review--------------------------

Add this Step Names line to models/medium_review.rb:

 %w(s1 s2 s3 s4 s5 s6 s7 s8 s9 s10 s11 s12 s13)

validates :medium_review_verdict, presence: true, if: -> { required_for_step?(:s12) }
validates :medium_review_sharing_mode, presence: true, if: -> { required_for_step?(:s13) }


Add those lines to controllers/medium_review_controller.rb: 
def medium_review_params
  params.require(:medium_review).permit(:id, :medium_is_blacklisted, :note_medium_is_blacklisted, :medium_failed_factcheck_before, :note_medium_failed_factcheck_before, :medium_has_poor_security, :note_medium_has_poor_security, :medium_whois_info_discrepency, :note_medium_whois_info_discrepency, :medium_hosting_discrepency, :note_medium_hosting_discrepency, :medium_is_biased, :note_medium_is_biased, :medium_is_poorly_ranked, :note_medium_is_poorly_ranked, :medium_is_not_liable, :note_medium_is_not_liable, :medium_lacks_flagging_means, :note_medium_lacks_flagging_mea

In [4]:
process_pages("src")

-------------------src_review--------------------------

Add this Step Names line to models/src_review.rb:

 %w(s1 s2 s3 s4 s5 s6 s7 s8 s9)

validates :src_review_verdict, presence: true, if: -> { required_for_step?(:s8) }
validates :src_review_sharing_mode, presence: true, if: -> { required_for_step?(:s9) }


Add those lines to controllers/src_review_controller.rb: 
def src_review_params
  params.require(:src_review).permit(:id, :src_lacks_proper_credentials, :note_src_lacks_proper_credentials, :src_failed_factcheck_before, :note_src_failed_factcheck_before, :src_has_poor_writing_history, :note_src_has_poor_writing_history, :src_supported_by_trolls, :note_src_supported_by_trolls, :src_difficult_to_locate, :note_src_difficult_to_locate, :src_other_problems, :note_src_other_problems, :src_review_verdict, :src_review_description, :note_src_review_description, :src_review_sharing_mode, :note_src_review_sharing_mode)
  end




Add this line to controllers/src_review/steps_controller.rb ins